In [ ]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
import httpx

from urllib.parse import urljoin

from will_it_saturate.server import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.core import FilesystemCreator, Epoch, Result
from will_it_saturate.client import HttpxClient, AioHttpClient, WrkClient
from will_it_saturate.persistence import SqliteRepository

<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

epoch.create_files()

result = BenchmarkResult(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

# result.elapsed = client.measure(epoch)

<IPython.core.display.Javascript object>

In [ ]:
server.start()

<IPython.core.display.Javascript object>

In [ ]:
server.started

True

<IPython.core.display.Javascript object>

In [ ]:
result.elapsed = client.measure(epoch)

measure


<IPython.core.display.Javascript object>

In [ ]:
result.elapsed

6.525339666999999

<IPython.core.display.Javascript object>

In [ ]:
result.bytes_per_second

57468272.785316154

<IPython.core.display.Javascript object>

# Remote Benchmark

In [ ]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

<IPython.core.display.Javascript object>

In [ ]:
control_base = "http://localhost:8001/"
control_epochs = urljoin(control_base, "epochs/")

r = httpx.post(control_epochs, json=epoch.dict_without_file_creator())
print(r.status_code, len(r.json()))

200 6


<IPython.core.display.Javascript object>

In [ ]:
server = FastAPIUvicornServer()

<IPython.core.display.Javascript object>

In [ ]:
server.dict()

{'name': 'fastAPI/uvicorn'}

<IPython.core.display.Javascript object>

In [ ]:
server = FastAPIUvicornServer()
control_servers = urljoin(control_base, "servers/")
r = httpx.post(control_servers, json=server.dict())
print(r.status_code, r.json)

200 <bound method Response.json of <Response [200 OK]>>


<IPython.core.display.Javascript object>

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

epoch.create_files()

result = BenchmarkResult(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

# result.elapsed = client.measure(epoch)